In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib.patches import Circle
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
from PyQt5.QtWidgets import (QApplication, QMainWindow, QWidget, QVBoxLayout,
                             QPushButton, QLabel, QComboBox, QFileDialog)
from PyQt5.QtWidgets import QHBoxLayout
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar

# 상수 및 설정 값 관리
class Config:
    WAFER_RADIUS = 150000
    SCALE_FACTOR = 1e-7
    SCALE_BAR_LENGTH = 30000
    SCALE_BAR_POSITION = 'lower center'
    SCALE_BAR_LENGTH_PIXELS = 30000
    SCALE_BAR_SIZE_VERTICAL = 500
    TEXT_POSITION_Y = -170000  # 텍스트 추가 위치

# 가로선과 세로선을 계산하는 함수 통합
def calculate_lines(center, pitch, max_die, min_die):
    lines = []
    current = center
    for _ in range(max_die + 2):
        lines.append(current)
        current += pitch
    current = center
    for _ in range(abs(min_die) + 1):
        current -= pitch
        lines.append(current)
    return lines

# 통계 계산 함수 (ABS(Mean) + 3sigma)
def calculate_statistics(values):
    mean_val = np.mean(values)
    sigma_val = np.std(values)
    m3s_val = np.abs(mean_val) + 3 * sigma_val
    m3s_nm = m3s_val * 1e3  # nm 단위로 변환
    return m3s_nm

# Overlay를 플롯하는 함수
def plot_overlay(ax, x, y, dx, dy, v_lines, h_lines, wafer_radius=Config.WAFER_RADIUS,
                 title='Wafer Vector Map', scale_factor=Config.SCALE_FACTOR,
                 show_statistics=True):
    quiver = ax.quiver(x, y, dx, dy, angles='xy', scale_units='xy', scale=scale_factor,
                       color='blue', label='Overlay Vectors',
                       width=0.0015, headwidth=3, headlength=3)

    ax.axvline(0, color='red', linewidth=1.0, label='Central X')
    ax.axhline(0, color='red', linewidth=1.0, label='Central Y')

    for vline in v_lines:
        ax.axvline(vline, color='black', linestyle='--', linewidth=0.8)
    for hline in h_lines:
        ax.axhline(hline, color='black', linestyle='--', linewidth=0.8)

    wafer_circle = Circle((0, 0), wafer_radius, color='green', fill=False,
                          linestyle='-', linewidth=2, label='Wafer Boundary')
    ax.add_patch(wafer_circle)

    scale_bar_length = Config.SCALE_BAR_LENGTH * scale_factor
    scale_bar_label = f'{scale_bar_length * 1e3:.1f}nm'

    fontprops = fm.FontProperties(size=10)
    scalebar = AnchoredSizeBar(ax.transData, Config.SCALE_BAR_LENGTH_PIXELS, scale_bar_label,
                               Config.SCALE_BAR_POSITION, pad=0.1, color='black',
                               frameon=False, size_vertical=Config.SCALE_BAR_SIZE_VERTICAL,
                               fontproperties=fontprops)
    ax.add_artist(scalebar)

    # 통계치 계산 및 텍스트 추가
    if show_statistics:
        mean_plus_3sigma_x = calculate_statistics(dx)
        mean_plus_3sigma_y = calculate_statistics(dy)
        ax.text(0, Config.TEXT_POSITION_Y,
                f'|m|+3s X: {mean_plus_3sigma_x:.2f} nm\n|m|+3s Y: {mean_plus_3sigma_y:.2f} nm',
                fontsize=10, color='red', ha='center')

    ax.set_xlabel('Wafer X Coordinate (wf_x)')
    ax.set_ylabel('Wafer Y Coordinate (wf_y)')
    ax.set_title(title)
    ax.axis('equal')
    ax.grid(False)

    return quiver

# 클릭한 플롯을 확대해서 보여주는 클래스
class EnlargedPlotWindow(QMainWindow):
    def __init__(self, x, y, u, v, title, v_lines, h_lines,
                 wafer_radius=Config.WAFER_RADIUS, scale_factor=Config.SCALE_FACTOR):
        super().__init__()
        self.setWindowTitle(title)
        self.setGeometry(100, 100, 800, 800)

        # 확대된 플롯을 생성
        fig, ax = plt.subplots(figsize=(8, 8))

        # plot_overlay 함수 사용
        plot_overlay(ax, x, y, u, v, v_lines, h_lines, wafer_radius, title, scale_factor)

        # Matplotlib FigureCanvas 생성
        self.canvas = FigureCanvas(fig)

        # Navigation Toolbar 추가
        self.toolbar = NavigationToolbar(self.canvas, self)

        layout = QVBoxLayout()
        layout.addWidget(self.toolbar)
        layout.addWidget(self.canvas)

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)

# 플롯을 보여주는 클래스
class PlotWindow(QMainWindow):
    def __init__(self, unique_id, df):
        super().__init__()
        self.setWindowTitle(f"Plot for {unique_id}")
        self.setGeometry(100, 100, 1200, 800)

        # 데이터 필터링
        df_lot = df[df['UNIQUE_ID'] == unique_id]

        # 데이터 추출
        data = df_lot.to_dict(orient='list')
        wf_x = data['wf_x']
        wf_y = data['wf_y']

        # 변수 목록
        variables = ['X_reg', 'Y_reg', 'pred_x', 'pred_y', 'residual_x', 'residual_y',
                     'psm_fit_x', 'psm_fit_y', 'residual_x_depsm', 'residual_y_depsm',
                     'cpe19p_pred_x', 'cpe19p_pred_y', 'cpe19p_resi_x', 'cpe19p_resi_y',
                     'ideal_psm_x', 'ideal_psm_y', 'delta_psm_x', 'delta_psm_y']

        # 스텝 피치 및 맵 시프트 추출
        step_pitch_x = df_lot['STEP_PITCH_X'].iloc[0]
        step_pitch_y = df_lot['STEP_PITCH_Y'].iloc[0]
        map_shift_x = df_lot['MAP_SHIFT_X'].iloc[0]
        map_shift_y = df_lot['MAP_SHIFT_Y'].iloc[0]
        start_left = -(step_pitch_x)/2 + map_shift_x
        start_bottom = -(step_pitch_y)/2 + map_shift_y
        max_die_x = int(df_lot['DieX'].max())
        min_die_x = int(df_lot['DieX'].min())
        max_die_y = int(df_lot['DieY'].max())
        min_die_y = int(df_lot['DieY'].min())

        self.vertical_lines = calculate_lines(start_left, step_pitch_x, max_die_x, min_die_x)
        self.horizontal_lines = calculate_lines(start_bottom, step_pitch_y, max_die_y, min_die_y)

        # 확대된 창들을 저장할 리스트
        self.enlarged_plot_windows = []

        # Figure 생성
        fig, axes = plt.subplots(4, 3, figsize=(12, 8))
        fig.suptitle(f'Visualizations for Lot {unique_id}', fontsize=16)

        self.quivers = []

        # 플롯 설정 정보를 리스트로 정의
        plot_configs = [
            {'ax': axes[0, 0], 'dx': data['X_reg'], 'dy': data['Y_reg'], 'title': 'Raw(X_reg,Y_reg)'},
            {'ax': axes[0, 1], 'dx': data['pred_x'], 'dy': data['pred_y'], 'title': 'OSR_Fitting(WK,RK)'},
            {'ax': axes[0, 2], 'dx': data['residual_x'], 'dy': data['residual_y'], 'title': 'Residual'},
            {'ax': axes[1, 0], 'dx': data['psm_fit_x'], 'dy': data['psm_fit_y'], 'title': 'PSM Input'},
            {'ax': axes[1, 1], 'dx': data['residual_x_depsm'], 'dy': data['residual_y_depsm'], 'title': 'Residual(Remove_PSM)'},
            {'ax': axes[1, 2], 'dx': data['cpe19p_pred_x'], 'dy': data['cpe19p_pred_y'], 'title': 'CPE 19para Fitting'},
            {'ax': axes[2, 0], 'dx': data['cpe19p_resi_x'], 'dy': data['cpe19p_resi_y'], 'title': 'CPE 19para Residual'},
            {'ax': axes[2, 1], 'dx': data['ideal_psm_x'], 'dy': data['ideal_psm_y'], 'title': 'Ideal PSM'},
            {'ax': axes[2, 2], 'dx': data['delta_psm_x'], 'dy': data['delta_psm_y'], 'title': 'Delta PSM'},
            {'ax': axes[3, 0], 'dx': data['delta_psm_x'], 'dy': [0]*len(data['delta_psm_y']), 'title': 'Delta PSM X'},
            {'ax': axes[3, 1], 'dx': [0]*len(data['delta_psm_x']), 'dy': data['delta_psm_y'], 'title': 'Delta PSM Y'},
            # 필요에 따라 더 추가 가능
        ]

        for config in plot_configs:
            quiver = plot_overlay(config['ax'], wf_x, wf_y, config['dx'], config['dy'],
                                  self.vertical_lines, self.horizontal_lines,
                                  title=config['title'])
            self.quivers.append((quiver, config['title']))

        # 사용되지 않는 서브플롯 숨기기 (예: axes[3,2])
        axes[3, 2].axis('off')

        # Matplotlib FigureCanvas 생성
        self.canvas = FigureCanvas(fig)

        # Navigation Toolbar 추가
        self.toolbar = NavigationToolbar(self.canvas, self)

        layout = QVBoxLayout()
        layout.addWidget(self.toolbar)
        layout.addWidget(self.canvas)

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)

        # 클릭 이벤트 연결
        self.canvas.mpl_connect('button_press_event', self.on_click)

    def on_click(self, event):
        for quiver, title in self.quivers:
            if event.inaxes and quiver.contains(event)[0]:
                # 클릭한 플롯을 확대하여 새 창으로 표시
                enlarged_window = EnlargedPlotWindow(
                    quiver.X, quiver.Y, quiver.U, quiver.V,
                    title=title,
                    v_lines=self.vertical_lines,
                    h_lines=self.horizontal_lines
                )
                self.enlarged_plot_windows.append(enlarged_window)
                enlarged_window.show()

# 메인 윈도우
class MainWindow(QMainWindow):
    def __init__(self, df):
        super().__init__()
        self.setWindowTitle("Unique ID Plot Viewer")
        self.setGeometry(100, 100, 400, 200)

        layout = QVBoxLayout()

        # 라벨 추가
        label = QLabel("Select a unique_id to view the plot:")
        layout.addWidget(label)

        # 콤보박스 추가
        self.combo_box = QComboBox()
        self.combo_box.addItems(df['UNIQUE_ID'].unique())
        layout.addWidget(self.combo_box)

        # 버튼 추가
        button = QPushButton("Show Plot")
        button.clicked.connect(self.show_plot)
        layout.addWidget(button)

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)

        # 데이터 저장
        self.df = df

    def show_plot(self):
        unique_id = self.combo_box.currentText()
        self.plot_window = PlotWindow(unique_id, self.df)
        self.plot_window.show()

# PyQt 애플리케이션 실행
def main():
    app = QApplication([])
    file_dialog = QFileDialog()
    file_path, _ = file_dialog.getOpenFileName(None, "Select CSV File", "", "CSV Files (*.csv)")
    if not file_path:
        return
    # CSV 데이터 로드
    df = pd.read_csv(file_path)

    main_window = MainWindow(df)
    main_window.show()
    app.exec_()

if __name__ == "__main__":
    main()



RuntimeError: wrapped C/C++ object of type FigureCanvasQTAgg has been deleted

<Figure size 782x742 with 1 Axes>

RuntimeError: wrapped C/C++ object of type FigureCanvasQTAgg has been deleted

<Figure size 782x742 with 1 Axes>

RuntimeError: wrapped C/C++ object of type FigureCanvasQTAgg has been deleted

...

<Figure size 782x742 with 1 Axes>

RuntimeError: wrapped C/C++ object of type FigureCanvasQTAgg has been deleted

<Figure size 1902x951 with 12 Axes>

RuntimeError: wrapped C/C++ object of type FigureCanvasQTAgg has been deleted

<Figure size 782x742 with 1 Axes>